In [1]:
import numpy as np
import pandas as pd
import os 

In [33]:
totimeslot = lambda x: ((int(x[-5:-3]) + (int(x[-8:-6])*60))//10)+1

def PrepareFeatureVector(setname, daynum):
    #Defining all names
    weatherfname = setname + 'weather_data/weather_data_2016-01-' + '{0:02}'.format(daynum)
    orderfname = setname + 'order_data/order_data_2016-01-' + '{0:02}'.format(daynum)
    poifname = setname + 'poi_data/poi_data'
    clustermapfname = setname + 'cluster_map/cluster_map'
    
    #Reading all files for the day given as argument
    orderdf = pd.read_csv(orderfname, sep='\t', header=None, na_filter=False, names=['OrderID', 'DriverID',
                          'PassengerID', 'StartRegionHash', 'DestRegionHash', 'Price', 'Time'], index_col = 0)
    
    weatherdf = pd.read_csv(weatherfname, sep='\t', header=None, na_filter=False, 
                 names=['time', 'weather', 'temperature', 'pm2.5'])
    
    poidf = pd.read_csv(poifname, names=[x for x in range(152)], index_col=0)
    clustermapdf = pd.read_csv(clustermapfname, sep='\t', names=['RegionHash', 'RegionID'], index_col = 0)
    
    #Mapping region hash values to problem-specific region codes 
    orderdf['StartRegionID'] = clustermapdf.ix[orderdf['StartRegionHash'].values].values[:,0]
    
    #Adding timeslot of each order 
    orderdf['Timeslot'] = orderdf['Time'].apply(totimeslot)
    
    #Adding weather condition to each order
    weatherdf['timeslot'] = weatherdf['time'].apply(totimeslot)
    orderdf['Weather'] = (((weatherdf.groupby('timeslot')['weather'].mean()).ix[orderdf['Timeslot'].values]).values).astype(int)
    
    #Adding temperature info, and pollution information. 
    orderdf['Temperature'] = (weatherdf.groupby('timeslot')['temperature'].mean()).ix[orderdf['Timeslot'].values].values 
    orderdf['PM2.5'] = (weatherdf.groupby('timeslot')['pm2.5'].mean()).ix[orderdf['Timeslot'].values].values
    
    #Adding Facility information
    return orderdf

def PrepareFeatureVectors(reponame):
    return None

for d in np.arange(1, 2):
    print 'testing for day = {}'.format(d)
    test=PrepareFeatureVector('./training_set/', d)

testing for day = 1


In [4]:
def GetPdDataFrame(reponame, colnames=[], indexcol = None):
    fnames = [os.path.join(reponame, x) for x in os.listdir(reponame) if x[0]!='.']
    out = None 
    print 'reading file =', 
    for i, fname in enumerate(fnames):
        print str(i+1)+',',
        if out is None: 
            out = pd.read_csv(fname, sep='\t', header=None, na_filter=False,
                              names=None if (colnames==[]) else colnames,
                             index_col = None if (indexcol) else indexcol)
        else:
            out = pd.concat((out, pd.read_csv(fname, sep='\t', header=None, na_filter=False,
                                             names=None if (colnames==[]) else colnames,
                                             index_col = None if (indexcol) else indexcol)))
    print 
    return out

def PrepareFeatVect(reponame):
    return None



In [14]:
#Defining all names
setname = './training_set/'
daynum = 1 
weatherfname = setname + 'weather_data/weather_data_2016-01-' + '{0:02}'.format(daynum)
orderfname = setname + 'order_data/order_data_2016-01-' + '{0:02}'.format(daynum)
poifname = setname + 'poi_data/poi_data'
clustermapfname = setname + 'cluster_map/cluster_map'

#Reading all files for the day given as argument
orderdf = pd.read_csv(orderfname, sep='\t', header=None, na_filter=False, names=['OrderID', 'DriverID',
                      'PassengerID', 'StartRegionHash', 'DestRegionHash', 'Price', 'Time'], index_col = 0)

weatherdf = pd.read_csv(weatherfname, sep='\t', header=None, na_filter=False, 
             names=['time', 'weather', 'temperature', 'pm2.5'])

poidf = pd.read_csv(poifname, names=[x for x in range(152)], index_col=0)
clustermapdf = pd.read_csv(clustermapfname, sep='\t', names=['RegionHash', 'RegionID'], index_col = 0)

In [15]:
#Mapping region hash values to problem-specific region codes 
orderdf['StartRegionID'] = clustermapdf.ix[orderdf['StartRegionHash'].values].values[:,0]

#Adding timeslot of each order 
orderdf['Timeslot'] = orderdf['Time'].apply(totimeslot)

In [16]:
orderdf.head() 

,DriverID,PassengerID,StartRegionHash,DestRegionHash,Price,Time,StartRegionID,Timeslot
OrderID,,,,,,,,
97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24.0,2016-01-01 13:37:23,23,82
92c3ac9251cc9b5aab90b114a1e363be,c077e0297639edcb1df6189e8cda2c3d,191a180f0a262aff3267775c4fac8972,82cc4851f9e4faa4e54309f8bb73fd7c,b05379ac3f9b7d99370d443cfd5dcc28,2.0,2016-01-01 09:47:54,8,59
abeefc3e2aec952468e2fd42a1649640,86dbc1b68de435957c61b5a523854b69,7029e813bb3de8cc73a8615e2785070c,fff4e8465d1e12621bc361276b6217cf,fff4e8465d1e12621bc361276b6217cf,9.0,2016-01-01 18:24:02,32,111
cb31d0be64cda3cc66b46617bf49a05c,4fadfa6eeaa694742de036dddf02b0c4,21dc133ac68e4c07803d1c2f48988a83,4b7f6f4e2bf237b6cc58f57142bea5c0,4b7f6f4e2bf237b6cc58f57142bea5c0,11.0,2016-01-01 22:13:27,13,134
139d492189ae5a933122c098f63252b3,NULL,26963cc76da2d8450d8f23fc357db987,fc34648599753c9e74ab238e9a4a07ad,87285a66236346350541b8815c5fae94,4.0,2016-01-01 17:00:06,27,103


In [21]:
weatherdf['timeslot'] = weatherdf['time'].apply(totimeslot)
orderdf['Weather'] = (((weatherdf.groupby('timeslot')['weather'].mean()).ix[orderdf['Timeslot'].values]).values).astype(int)

In [22]:
orderdf.head() 

,DriverID,PassengerID,StartRegionHash,DestRegionHash,Price,Time,StartRegionID,Timeslot,Weather
OrderID,,,,,,,,,
97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24.0,2016-01-01 13:37:23,23,82,2
92c3ac9251cc9b5aab90b114a1e363be,c077e0297639edcb1df6189e8cda2c3d,191a180f0a262aff3267775c4fac8972,82cc4851f9e4faa4e54309f8bb73fd7c,b05379ac3f9b7d99370d443cfd5dcc28,2.0,2016-01-01 09:47:54,8,59,2
abeefc3e2aec952468e2fd42a1649640,86dbc1b68de435957c61b5a523854b69,7029e813bb3de8cc73a8615e2785070c,fff4e8465d1e12621bc361276b6217cf,fff4e8465d1e12621bc361276b6217cf,9.0,2016-01-01 18:24:02,32,111,2
cb31d0be64cda3cc66b46617bf49a05c,4fadfa6eeaa694742de036dddf02b0c4,21dc133ac68e4c07803d1c2f48988a83,4b7f6f4e2bf237b6cc58f57142bea5c0,4b7f6f4e2bf237b6cc58f57142bea5c0,11.0,2016-01-01 22:13:27,13,134,2
139d492189ae5a933122c098f63252b3,NULL,26963cc76da2d8450d8f23fc357db987,fc34648599753c9e74ab238e9a4a07ad,87285a66236346350541b8815c5fae94,4.0,2016-01-01 17:00:06,27,103,2


In [34]:
test.head() 

,DriverID,PassengerID,StartRegionHash,DestRegionHash,Price,Time,StartRegionID,Timeslot,Weather,Temperature,PM2.5
OrderID,,,,,,,,,,,
97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24.0,2016-01-01 13:37:23,23,82,2,10.5,232.0
92c3ac9251cc9b5aab90b114a1e363be,c077e0297639edcb1df6189e8cda2c3d,191a180f0a262aff3267775c4fac8972,82cc4851f9e4faa4e54309f8bb73fd7c,b05379ac3f9b7d99370d443cfd5dcc28,2.0,2016-01-01 09:47:54,8,59,2,4.0,186.0
abeefc3e2aec952468e2fd42a1649640,86dbc1b68de435957c61b5a523854b69,7029e813bb3de8cc73a8615e2785070c,fff4e8465d1e12621bc361276b6217cf,fff4e8465d1e12621bc361276b6217cf,9.0,2016-01-01 18:24:02,32,111,2,10.0,146.0
cb31d0be64cda3cc66b46617bf49a05c,4fadfa6eeaa694742de036dddf02b0c4,21dc133ac68e4c07803d1c2f48988a83,4b7f6f4e2bf237b6cc58f57142bea5c0,4b7f6f4e2bf237b6cc58f57142bea5c0,11.0,2016-01-01 22:13:27,13,134,2,8.0,128.0
139d492189ae5a933122c098f63252b3,NULL,26963cc76da2d8450d8f23fc357db987,fc34648599753c9e74ab238e9a4a07ad,87285a66236346350541b8815c5fae94,4.0,2016-01-01 17:00:06,27,103,2,11.0,154.0


In [26]:
orderdf['Temperature'] = (weatherdf.groupby('timeslot')['temperature'].mean()).ix[orderdf['Timeslot'].values].values 
orderdf['PM2.5'] = (weatherdf.groupby('timeslot')['pm2.5'].mean()).ix[orderdf['Timeslot'].values].values

In [27]:
orderdf.head() 

,DriverID,PassengerID,StartRegionHash,DestRegionHash,Price,Time,StartRegionID,Timeslot,Weather,Temperature,PM2.5
OrderID,,,,,,,,,,,
97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24.0,2016-01-01 13:37:23,23,82,2,10.5,232.0
92c3ac9251cc9b5aab90b114a1e363be,c077e0297639edcb1df6189e8cda2c3d,191a180f0a262aff3267775c4fac8972,82cc4851f9e4faa4e54309f8bb73fd7c,b05379ac3f9b7d99370d443cfd5dcc28,2.0,2016-01-01 09:47:54,8,59,2,4.0,186.0
abeefc3e2aec952468e2fd42a1649640,86dbc1b68de435957c61b5a523854b69,7029e813bb3de8cc73a8615e2785070c,fff4e8465d1e12621bc361276b6217cf,fff4e8465d1e12621bc361276b6217cf,9.0,2016-01-01 18:24:02,32,111,2,10.0,146.0
cb31d0be64cda3cc66b46617bf49a05c,4fadfa6eeaa694742de036dddf02b0c4,21dc133ac68e4c07803d1c2f48988a83,4b7f6f4e2bf237b6cc58f57142bea5c0,4b7f6f4e2bf237b6cc58f57142bea5c0,11.0,2016-01-01 22:13:27,13,134,2,8.0,128.0
139d492189ae5a933122c098f63252b3,NULL,26963cc76da2d8450d8f23fc357db987,fc34648599753c9e74ab238e9a4a07ad,87285a66236346350541b8815c5fae94,4.0,2016-01-01 17:00:06,27,103,2,11.0,154.0
